In [51]:
# Import the requests library.
import requests

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the API key.
from config import weather_api_key

from citipy import citipy

In [52]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [53]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [54]:
# Create a list for holding the cities. 
cities = []


# Identify nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to our cities list
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count
len(cities)

744

In [55]:
# Import the requests library.
import requests

# Import the time library.
import time

# Impot the datetime module from the datetime library. 
from datetime import datetime

In [56]:
# Starting URL for Weather May API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [57]:
# List of city data
city_data = []

# Print to logger
print("Beginning Data Retrieval        ")
print("--------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # print(city_url)
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the cities
    try: 
        # response = requests.get(city_url)
        # print(response.status_code)
        # city_weather = response.json()
        city_weather = requests.get(city_url).json()
        
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["weather"][0]["description"]
        
            # Append the City information into city_data list
        city_data.append({"City": city.title(),
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Current Description": city_weather_description,
                         "Humidity": city_humidity,
                         "Cloudiness": city_clouds, 
                         "Wind Speed": city_wind,
                         "Country": city_country})
        
    # If an error is experinced, skip the city
    # To see what kind of error...
    # except BaseException as err: 
        # print(err)
        
    except: 
        print("City not found. Skipping...")
        pass
    
# Indicate that Data loading is complete
print("------------------------------")
print("Data Retrieval Complete       ")
print("------------------------------")
        

Beginning Data Retrieval        
--------------------------------
Processing Record 1 of Set 1 | butaritari
Processing Record 2 of Set 1 | new norfolk
Processing Record 3 of Set 1 | torbay
Processing Record 4 of Set 1 | albany
Processing Record 5 of Set 1 | lompoc
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | hermanus
Processing Record 8 of Set 1 | qaanaaq
Processing Record 9 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 10 of Set 1 | ponta do sol
Processing Record 11 of Set 1 | jamestown
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | caravelas
Processing Record 14 of Set 1 | puerto ayora
Processing Record 15 of Set 1 | san patricio
Processing Record 16 of Set 1 | ponta delgada
Processing Record 17 of Set 1 | kitimat
Processing Record 18 of Set 1 | manaus
Processing Record 19 of Set 1 | berdigestyakh
Processing Record 20 of Set 1 | mataura
Processing Record 21 of Set 1 | yellowknife
Processing Record 22 of Set 1 | pu

Processing Record 34 of Set 4 | moranbah
Processing Record 35 of Set 4 | mahajanga
Processing Record 36 of Set 4 | jardim
Processing Record 37 of Set 4 | imeni poliny osipenko
Processing Record 38 of Set 4 | roswell
Processing Record 39 of Set 4 | havoysund
Processing Record 40 of Set 4 | sur
Processing Record 41 of Set 4 | cabo san lucas
Processing Record 42 of Set 4 | eyl
Processing Record 43 of Set 4 | saldanha
Processing Record 44 of Set 4 | gushikawa
Processing Record 45 of Set 4 | arraial do cabo
Processing Record 46 of Set 4 | petrolina
Processing Record 47 of Set 4 | toluviejo
Processing Record 48 of Set 4 | bubaque
Processing Record 49 of Set 4 | winburg
Processing Record 50 of Set 4 | cap malheureux
Processing Record 1 of Set 5 | seddon
Processing Record 2 of Set 5 | altayskoye
Processing Record 3 of Set 5 | polunochnoye
Processing Record 4 of Set 5 | simi
Processing Record 5 of Set 5 | saint-pierre
Processing Record 6 of Set 5 | grand-lahou
Processing Record 7 of Set 5 | kom

Processing Record 24 of Set 8 | takoradi
Processing Record 25 of Set 8 | boston
Processing Record 26 of Set 8 | guiren
Processing Record 27 of Set 8 | basarabeasca
Processing Record 28 of Set 8 | santa maria
Processing Record 29 of Set 8 | dodge city
Processing Record 30 of Set 8 | zhuhai
Processing Record 31 of Set 8 | padang
Processing Record 32 of Set 8 | sao filipe
Processing Record 33 of Set 8 | aklavik
Processing Record 34 of Set 8 | gorno-chuyskiy
City not found. Skipping...
Processing Record 35 of Set 8 | riyadh
Processing Record 36 of Set 8 | kaduqli
Processing Record 37 of Set 8 | puerto leguizamo
Processing Record 38 of Set 8 | plettenberg bay
Processing Record 39 of Set 8 | bilibino
Processing Record 40 of Set 8 | nueva italia
Processing Record 41 of Set 8 | belaya gora
Processing Record 42 of Set 8 | tempio pausania
Processing Record 43 of Set 8 | benxi
Processing Record 44 of Set 8 | saint-augustin
Processing Record 45 of Set 8 | cangas
Processing Record 46 of Set 8 | hob

Processing Record 12 of Set 12 | tera
Processing Record 13 of Set 12 | jalu
Processing Record 14 of Set 12 | bonavista
Processing Record 15 of Set 12 | fare
Processing Record 16 of Set 12 | dundwaraganj
Processing Record 17 of Set 12 | clarksdale
Processing Record 18 of Set 12 | huangchuan
Processing Record 19 of Set 12 | bukachacha
Processing Record 20 of Set 12 | uray
Processing Record 21 of Set 12 | straumen
Processing Record 22 of Set 12 | tilichiki
Processing Record 23 of Set 12 | aranos
Processing Record 24 of Set 12 | smoky lake
Processing Record 25 of Set 12 | manakara
Processing Record 26 of Set 12 | kamaishi
Processing Record 27 of Set 12 | umba
Processing Record 28 of Set 12 | aykhal
Processing Record 29 of Set 12 | hualmay
Processing Record 30 of Set 12 | denpasar
Processing Record 31 of Set 12 | oriximina
Processing Record 32 of Set 12 | shingu
Processing Record 33 of Set 12 | adrar
Processing Record 34 of Set 12 | amapa
Processing Record 35 of Set 12 | kiunga
Processing R

In [58]:
len(city_data)

685

In [59]:
# Convert array of dictionaries to a pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [60]:
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Current Description,Humidity,Cloudiness,Wind Speed,Country
0,Butaritari,3.0707,172.7902,80.91,light rain,80,99,10.85,KI
1,New Norfolk,-42.7826,147.0587,43.92,few clouds,87,23,1.01,AU
2,Torbay,47.6666,-52.7314,63.37,few clouds,34,20,6.91,CA
3,Albany,42.6001,-73.9662,91.67,clear sky,54,0,5.01,US
4,Lompoc,34.6391,-120.4579,76.78,clear sky,58,0,12.66,US
5,Ushuaia,-54.8000,-68.3000,42.46,clear sky,70,0,6.91,AR
6,Hermanus,-34.4187,19.2345,57.74,scattered clouds,76,37,5.39,ZA
7,Qaanaaq,77.4840,-69.3632,19.78,overcast clouds,80,100,2.55,GL
8,Ponta Do Sol,32.6667,-17.1000,70.43,few clouds,82,22,2.42,PT
9,Jamestown,42.0970,-79.2353,83.84,clear sky,57,0,12.66,US


In [61]:
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Current Description', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country'],
      dtype='object')

In [62]:
# Reorder the columns in the order you want them to appear.
new_column_order = ["City", "Country", "Lat", "Lng", 
                   "Max Temp", "Humidity", "Cloudiness", 
                   "Wind Speed", "Current Description"]

# Assign a new or the same DataFrame the new column order.
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Butaritari,KI,3.0707,172.7902,80.91,80,99,10.85,light rain
1,New Norfolk,AU,-42.7826,147.0587,43.92,87,23,1.01,few clouds
2,Torbay,CA,47.6666,-52.7314,63.37,34,20,6.91,few clouds
3,Albany,US,42.6001,-73.9662,91.67,54,0,5.01,clear sky
4,Lompoc,US,34.6391,-120.4579,76.78,58,0,12.66,clear sky


In [63]:
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [64]:
# Create the output File (CSV)
output_data_file = "WeatherPy_Database.csv"
# Export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")